In [2]:
import pandas as pd

# Define the bucket and file names
bucket_name = 'mimicivliza'  # Replace with your bucket name
KG_Data = f's3://{bucket_name}/grouped_combined_df.csv'

# Load the files
KG_Data = pd.read_csv(KG_Data)

# Display the data
KG_Data.head(5)

,subject_id,icd_code_procedures,long_title_procedures,icd_code_diagnosis,long_title_diagnosis,drug,prod_strength
0,10000032,['5491'],['Percutaneous abdominal drainage'],"['2761', 'V08', '2875', '3051', '5715', '496',...","['Hyposmolality and/or hyponatremia', 'Asympto...","Tiotropium Bromide, Albuterol 0.083% Neb Soln,...","18mcg Capsule-Inhalation Device, 0.083%;3mL Vi..."
1,10001217,"['0331', '0139', '3897']","['Spinal tap', 'Other incision of brain', 'Cen...","['V168', 'V161', '04109', '3485', '4019', '348...",['Family history of other specified malignant ...,"Vial, Bag, OxycoDONE (Immediate Release) , *NF...","Send Vial, 50 mL Bag, 5mg Tablet, 1g Vial, 500..."
2,10001725,"['7094', '7055', '8696']","['Insertion of biological graft', 'Repair of r...","['49390', '7291', '6185', '56400', '5641', 'E9...","['Asthma, unspecified type, unspecified', 'Mya...","Zolpidem Tartrate, Bag, Lorazepam, Ondansetron...","5mg Tablet, 50 mL Bag, 1mg Tablet, 0.5mg Table..."
3,10002428,"['9390', '0DJD8ZZ', '7915', '9672', '9604', '3...","['Non-invasive mechanical ventilation', 'Inspe...","['4019', '29281', '4240', '00845', '1122', '71...","['Unspecified essential hypertension', 'Drug-i...","Insulin, Multivitamins, Neutra-Phos, Oxycodone...","Dummy Package for Sliding Scale, 1 Tablet, Pow..."
4,10002495,"['4A023N7', '027034Z', '5A02210', '4A023N6', '...",['Measurement of Cardiac Sampling and Pressure...,"['R310', 'N179', 'Z7901', 'Z86718', 'I214', 'A...","['Gross hematuria', 'Acute kidney failure, uns...","Insulin, Bag, Lidocaine Jelly 2% (Urojet), Ato...","100 Units / mL - 10 mL Vial, 50 mL Bag, 5mL Ur..."


In [5]:
print(KG_Data['long_title_procedures'].iloc[6])

['Other radiotherapeutic procedure', 'Colonoscopy', 'Biopsy of anus', 'Insertion of Endotracheal Airway into Trachea, Via Natural or Artificial Opening', 'Fluoroscopy of Left Kidney, Ureter and Bladder using Other Contrast', 'Respiratory Ventilation, 24-96 Consecutive Hours', 'Ultrasonography of Superior Vena Cava, Guidance', 'Drainage of Right Pleural Cavity with Drainage Device, Percutaneous Approach', 'Insertion of Infusion Device into Superior Vena Cava, Percutaneous Approach', 'Removal of Intraluminal Device from Ureter, Via Natural or Artificial Opening Endoscopic', 'Fluoroscopy of Right Kidney, Ureter and Bladder using Other Contrast', 'Dilation of Right Ureter with Intraluminal Device, Via Natural or Artificial Opening Endoscopic', 'Introduction of Nutritional Substance into Upper GI, Via Natural or Artificial Opening', 'Inspection of Bladder, Via Natural or Artificial Opening Endoscopic', 'Attachment of pedicle or flap graft to other sites', 'Other lysis of peritoneal adhesion

In [17]:
print(type(KG_Data['long_title_procedures'].iloc[6]))


<class 'list'>


In [19]:
print(KG_Data['long_title_diagnosis'].apply(type).value_counts())


long_title_diagnosis
<class 'list'>    92
Name: count, dtype: int64


### Importing Necessary Libraries

In [8]:
!pip install --upgrade --quiet langchain neo4j langchain

In [10]:
!pip install --quiet langchain-community

### Neo4j Credentials

In [30]:
## Graphdb configuration
NEO4J_URI=""
NEO4J_USERNAME=""
NEO4J_PASSWORD=""

import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

from langchain_community.graphs import Neo4jGraph
neo4j_graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [12]:
# Cypher query to count nodes and relationships
check_nodes_query = "MATCH (n) RETURN count(n) AS node_count"
check_relationships_query = "MATCH ()-[r]->() RETURN count(r) AS relationship_count"

# Execute the queries
node_count = neo4j_graph.query(check_nodes_query)
relationship_count = neo4j_graph.query(check_relationships_query)

node_count, relationship_count

([{'node_count': 68}], [{'relationship_count': 0}])

In [36]:
from langchain_community.graphs import Neo4jGraph

# Neo4j connection setup
NEO4J_URI = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""

# Initialize the Neo4jGraph object
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

# Test with single row
test_query = """
MERGE (p:Patient {id: $patient_id})
RETURN p
"""

# Try creating just a single patient node
try:
    # Take first row from dataframe
    test_row = KG_Data.iloc[0]
    
    # Test simple patient creation
    result = neo4j_graph.query(test_query, {"patient_id": int(test_row['subject_id'])})
    print("Test successful! Created patient node.")
    print(f"Patient ID: {test_row['subject_id']}")
    
    # If successful, try one complete record
    single_record = KG_Data.iloc[0:1].to_dict(orient='records')[0]
    neo4j_graph.query(cypher_query, {"patients": [single_record]})
    print("Successfully created complete record with all relationships!")
    
except Exception as e:
    print(f"Error occurred: {e}")
    print("\nDebug information:")
    print(f"Data type of subject_id: {type(test_row['subject_id'])}")
    print(f"Row data: {test_row.to_dict()}")

Test successful! Created patient node.
Patient ID: 10000032
Successfully created complete record with all relationships!


In [38]:
# Query to delete all nodes and relationships
delete_all_query = "MATCH (n) DETACH DELETE n"

# Execute the query
neo4j_graph.query(delete_all_query)

# Verify that everything has been deleted
node_count = neo4j_graph.query("MATCH (n) RETURN count(n) AS node_count")
relationship_count = neo4j_graph.query("MATCH ()-[r]->() RETURN count(r) AS relationship_count")

node_count, relationship_count


([{'node_count': 0}], [{'relationship_count': 0}])

In [42]:
from tqdm import tqdm
from langchain_community.graphs import Neo4jGraph
import pandas as pd

# Neo4j connection setup
NEO4J_URI = "neo4j+s://8a886660.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "9FYDkCTM2Vq4qxWFFwik0uYP6BJ-fReP9XOYj-oDqZ4"

# Initialize the Neo4jGraph object
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

# First, let's look at the data
print("Data types of columns:")
print(KG_Data.dtypes)

print("\nFirst row sample:")
first_row = KG_Data.iloc[0]
print("\nSubject ID:", first_row['subject_id'])
print("\nProcedures code:", first_row['icd_code_procedures'])
print("\nProcedures title:", first_row['long_title_procedures'])
print("\nDrug:", first_row['drug'])

# Let's try with just one record first
test_query = """
MERGE (p:Patient {id: $patient_id})
RETURN p
"""

try:
    # Try with just one patient
    result = neo4j_graph.query(test_query, {"patient_id": int(KG_Data.iloc[0]['subject_id'])})
    print("\nTest patient creation successful!")
    
    # Now try one complete record
    first_record = KG_Data.iloc[0:1].to_dict(orient='records')[0]
    print("\nFirst record data structure:")
    print(first_record)
    
except Exception as e:
    print(f"\nError: {e}")

Data types of columns:
subject_id                int64
icd_code_procedures      object
long_title_procedures    object
icd_code_diagnosis       object
long_title_diagnosis     object
drug                     object
prod_strength            object
dtype: object

First row sample:

Subject ID: 10000032

Procedures code: ['5491']

Procedures title: ['Percutaneous abdominal drainage']

Drug: Tiotropium Bromide, Albuterol 0.083% Neb Soln, Sulfameth/Trimethoprim DS, Furosemide, Heparin, Ipratropium Bromide Neb, Morphine Sulfate, Albumin 25% (12.5g / 50mL), Acetaminophen, Bisacodyl, Calcium Carbonate, Dextrose 50%, Lactulose, Nicotine Patch, 0.9% Sodium Chloride, Raltegravir, Rifaximin, Senna, Docusate Sodium (Liquid), TraZODone, Influenza Vaccine Quadrivalent, Emtricitabine-Tenofovir (Truvada), Calcium Gluconate, Spironolactone, Sodium Polystyrene Sulfonate, Potassium Chloride, Insulin (Regular) for Hyperkalemia, Sodium Chloride 0.9%  Flush, Zolpidem Tartrate, TraMADOL (Ultram), Fluticasone 

In [ ]:
from tqdm import tqdm
from langchain_community.graphs import Neo4jGraph

# Neo4j connection setup
NEO4J_URI = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""

# Initialize the Neo4jGraph object
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

# Updated Cypher query with correct indexOf function
cypher_query = """
UNWIND $patients AS patient
// Create Patient nodes
MERGE (p:Patient {id: patient.subject_id})

// Create Procedure nodes and relationships
WITH patient, p
UNWIND patient.icd_code_procedures AS procedure_code
WITH patient, p, procedure_code, patient.long_title_procedures AS procedure_titles
WITH patient, p, procedure_code, procedure_titles[0] AS procedure_title
WHERE procedure_code IS NOT NULL
MERGE (proc:Procedure {code: procedure_code})
ON CREATE SET proc.title = procedure_title
MERGE (p)-[:HAS_PROCEDURE]->(proc)

// Create Diagnosis nodes and relationships
WITH patient, p
UNWIND patient.icd_code_diagnosis AS diagnosis_code
WITH patient, p, diagnosis_code, patient.long_title_diagnosis AS diagnosis_titles
WITH patient, p, diagnosis_code, diagnosis_titles[0] AS diagnosis_title
WHERE diagnosis_code IS NOT NULL
MERGE (diag:Diagnosis {code: diagnosis_code})
ON CREATE SET diag.title = diagnosis_title
MERGE (p)-[:HAS_DIAGNOSIS]->(diag)

// Create Drug nodes and relationships (split the comma-separated string)
WITH patient, p
UNWIND split(patient.drug, ',') AS drug_name
WITH patient, p, trim(drug_name) AS drug_name
WHERE drug_name IS NOT NULL
MERGE (drug:Drug {name: drug_name})
MERGE (p)-[:PRESCRIBED]->(drug)
"""

# Batch size
batch_size = 5

# Insert data into Neo4j in batches with progress tracking
try:
    for i in tqdm(range(0, len(KG_Data), batch_size), desc="Inserting patients into Neo4j"):
        batch = KG_Data.iloc[i:i + batch_size].to_dict(orient='records')
        try:
            neo4j_graph.query(cypher_query, {"patients": batch})
            print(f"\nSuccessfully processed batch {i//batch_size + 1} of {(len(KG_Data) + batch_size - 1)//batch_size}")
            
            # Print intermediate counts every 5 batches
            if (i/batch_size) % 5 == 0:
                counts = neo4j_graph.query("""
                    MATCH (n)
                    WITH 
                        count(n) as total,
                        count(n:Patient) as patients,
                        count(n:Procedure) as procedures,
                        count(n:Diagnosis) as diagnoses,
                        count(n:Drug) as drugs
                    RETURN total, patients, procedures, diagnoses, drugs
                """)
                print("\nCurrent counts:", counts)
                
        except Exception as e:
            print(f"\nError processing batch starting at index {i}: {str(e)}")
            continue

    # Final counts
    final_counts = neo4j_graph.query("""
        MATCH (n)
        WITH 
            count(n) as total,
            count(n:Patient) as patients,
            count(n:Procedure) as procedures,
            count(n:Diagnosis) as diagnoses,
            count(n:Drug) as drugs
        RETURN total, patients, procedures, diagnoses, drugs
    """)
    print("\nFinal counts:", final_counts)

except Exception as e:
    print(f"\nCritical error occurred: {str(e)}")

Inserting patients into Neo4j:   0%|          | 0/19 [00:00<?, ?it/s]


Successfully processed batch 1 of 19


Inserting patients into Neo4j:   5%|▌         | 1/19 [00:13<04:07, 13.74s/it]


Current counts: [{'total': 339, 'patients': 339, 'procedures': 339, 'diagnoses': 339, 'drugs': 339}]


Inserting patients into Neo4j:  11%|█         | 2/19 [01:15<11:54, 42.04s/it]

...


Successfully processed batch 17 of 19


Inserting patients into Neo4j:  95%|█████████▍| 18/19 [15:44<01:39, 99.25s/it]


Successfully processed batch 18 of 19


Inserting patients into Neo4j: 100%|██████████| 19/19 [17:21<00:00, 54.81s/it]


Successfully processed batch 19 of 19

Final counts: [{'total': 2487, 'patients': 2487, 'procedures': 2487, 'diagnoses': 2487, 'drugs': 2487}]


In [46]:
# Test query to check current data structure
check_query = """
MATCH (n)
WITH labels(n) as nodeTypes, count(n) as count
RETURN nodeTypes, count
ORDER BY count DESC
"""

try:
    structure = neo4j_graph.query(check_query)
    print("Node structure:", structure)
    
    # Check relationships
    rel_query = """
    MATCH ()-[r]->()
    RETURN type(r) as relationship_type, count(r) as count
    ORDER BY count DESC
    """
    relationships = neo4j_graph.query(rel_query)
    print("\nRelationship counts:", relationships)
    
except Exception as e:
    print(f"Error checking structure: {e}")

Node structure: [{'nodeTypes': ['Diagnosis'], 'count': 1425}, {'nodeTypes': ['Drug'], 'count': 618}, {'nodeTypes': ['Procedure'], 'count': 352}, {'nodeTypes': ['Patient'], 'count': 92}]

Relationship counts: [{'relationship_type': 'PRESCRIBED', 'count': 5334}, {'relationship_type': 'HAS_DIAGNOSIS', 'count': 3061}, {'relationship_type': 'HAS_PROCEDURE', 'count': 608}]


In [48]:
# Total nodes and edges query
total_query = """
MATCH (n)
WITH count(n) as nodes
MATCH ()-[r]->()
RETURN nodes as total_nodes, count(r) as total_edges
"""

try:
    totals = neo4j_graph.query(total_query)
    print("Total counts:", totals)
except Exception as e:
    print(f"Error: {e}")

Total counts: [{'total_nodes': 2487, 'total_edges': 9003}]
